In [ ]:
library("httr")
library("jsonlite")
library("ggplot2")
library('RJSplot')
library('dplyr')

In [ ]:
base <- "193.175.238.88:8080/api/politicians/"
call1 <- paste(base,"all/", sep="")
get_politicians_df <- as.data.frame(fromJSON(content(GET(call1), "text", encoding="UTF-8"), flatten = TRUE))
# In the table 'get_politicians_df' you can see all the extracted politicians with respected twitter, facebook and wikipedia ids.

tables_rjs(get_politicians_df, jupyter = TRUE)

In [ ]:
# Please choose a politician from the table above and input the `politician_id` and `aggregation` variables below:
politician_id <- "27"
aggregation <- "month" # aggregation can be by day ('day'), week ('week') or month('month')

call_chobs <- paste(base,"wikipedia/chobs/politicians/", politician_id, "/?aggregate_by=", aggregation, sep="")
chobs = as.data.frame(fromJSON(content(GET(call_chobs), "text", encoding="UTF-8"), flatten = TRUE))

#extracting revisions, conflicts and undos via wikiwho API
base_ww <- "https://www.wikiwho.net/de/edit_persistence/v1.0.0-beta/page/"
wiki_id <- get_politicians_df$wp_ids[get_politicians_df$politician_id == politician_id]
wiki = as.data.frame(fromJSON(content(GET(paste(base_ww, wiki_id, "/", sep="")), "text", encoding="UTF-8"), flatten = TRUE))
wiki$undos <- wiki$editions.dels + wiki$editions.reins
wiki <- summarise(group_by(wiki, editions.year_month), undos = sum(undos),revisions = sum(editions.revisions), conflicts = sum(editions.conflicts))

merged = merge(chobs, wiki, by.x='labels', by.y='editions.year_month')

In [ ]:
#plotting
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = values, color="Change objects", group=response_type)) +
  geom_line(aes(y = revisions, color="Revisions", group=response_type)) +
  labs(title = "Wikipedia (chobs and revisions)", x = "Date", y = "N") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank()) +
  scale_x_date(date_breaks = "1 year")

In [ ]:
#plotting
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = merged, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = undos, color="Undos", group=response_type)) +
  geom_line(aes(y = conflicts, color="Conflicts", group=response_type)) +
  labs(title = "Wikipedia (undos and conflicts)", x = "Date", y = "N") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank()) +
  scale_x_date(date_breaks = "1 year")